# dm+d to ATC DDD linkage

To enable linking SCMD data to WHO DDDs we need to be able to link reliably from VMP code to an ATC code + a form in the WHO format.
The NHS BSA produce a lookup file to link individual VMP codes to ATC codes (using a one-to-one relationship).
In the same file the NHS BSA also provide some pre-specified DDDs - although the methodology behind this and the completeness is unclear.
This notebook sets out to set out a method to reliably link VMPs to WHO DDDs.

In [1]:
import pandas as pd
import requests
from ebmdatalab import bq
import os

### Converting route

The WHO DDD files have different DDDs for different routes of administration - the route is identified using the ```adm_code```.

We can show a list of distinct `adm_code` used in the WHO DDD files.

In [2]:
sql = f"""
    SELECT DISTINCT
      adm_code,
      COUNT(atc_code) AS count
    FROM `ebmdatalab.scmd.ddd`
    GROUP BY adm_code
    ORDER BY count DESC
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'unique_adm_code.csv')

# Use the cached_read function from the bq library to run the query.
unique_adm_code = bq.cached_read(sql, csv_path=csv_path)

unique_adm_code

,adm_code,count
0,O,1642
1,P,782
2,R,79
3,N,38
4,V,37
5,Inhal.solution,25
6,Inhal.powder,23
7,Inhal.aerosol,23
8,NaN,17
9,SL,16


The dm+d also holds similar data on routes. The `ontologyformroute` table has the most detailed information with a description of the formulation, then a full stop, then the route - for example - `tablet.oral`

We can show a list of distinct `ontologyformroute` descriptions:

In [3]:
sql = f"""
    SELECT DISTINCT
      ofr.descr
    FROM `ebmdatalab.dmd.vmp` vmp
    LEFT JOIN `ebmdatalab.dmd.ont` ont ON vmp.id = ont.vmp
    LEFT JOIN `ebmdatalab.dmd.ontformroute` ofr ON ont.form = ofr.cd    
    ORDER BY ofr.descr ASC
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'unique_ofr.csv')

# Use the cached_read function from the bq library to run the query.
unique_ofr = bq.cached_read(sql, csv_path=csv_path)

unique_ofr

,descr
0,NaN
1,bladderirrigation.intravesical
2,capsule.oral
3,capsule.vaginal
4,capsulechewable.oral
...,...
329,vapourinhalationimpregnatedpad.nasal
330,vapourinhalationliquid.inhalation
331,vapourinhalationliquid.nasal
332,vapourinhalationointment.nasal


To link between the WHO `adm_code` and the dm+d `ontologyformroute` we need to create a map.

Use of SQL code to do this makes the methodology clearer and easy to adapt in future.

Matching for many routes can simply be done based on matching the route part of the dm+d `ontologyformroute` to the appropriate `adm_code` for example matching all `.oral` products to the `adm_code` `O`

A small number require more careful matching, for example, looking at the `implant` `adm_code` we can see this actually relates to implants which are administered by subcutaneous injection - so we need to specifically match `implant.subcutaneous` to `implant` while the rest of the `subcutaneous` route will be matched to `P`.

There are a large number of routes in the dm+d which don't have DDDs defined by the WHO. We can disguard these from matching. A small percentage of these may be able to be matched at a later stage following a closer look at missing data.

In [4]:
sql = f"""
    SELECT DISTINCT
      ofr.descr AS dmd_ofr,
      SPLIT(ofr.descr, '.')[OFFSET(1)] AS dmd_r,
      CASE
        WHEN ofr.descr = 'implant.subcutaneous' THEN 'implant'
        WHEN ofr.descr = 'pressurizedinhalation.inhalation' THEN 'Inhal.aerosol'
        WHEN ofr.descr = 'powderinhalation.inhalation' THEN 'Inhal.powder'
        WHEN ofr.descr = 'solutionnebuliser.inhalation' THEN 'Inhal.solution'
        WHEN ofr.descr = 'dispersionnebuliser.inhalation' THEN 'Inhal.solution' -- amikacin liposomal
        WHEN ofr.descr = 'inhalationsolution.inhalation' THEN 'Inhal.solution' -- mist inhalers - e.g. tiotropium respimat
        WHEN ofr.descr = 'suspensionnebuliser.inhalation' THEN 'Inhal.solution' -- steroid nebs
        WHEN ofr.descr = 'medicatedchewing-gum.oromucosal' THEN 'Chewing gum' -- nicotine
        WHEN ofr.descr = 'solution.oromucosal' THEN 'SL' -- midazolam oromucosal
        WHEN ofr.descr = 'tabletsublingual.oromucosal' THEN 'SL'
        WHEN ofr.descr = 'lozenge.oromucosal' THEN 'SL' -- fentanyl lozenge
        WHEN ofr.descr = 'lyophilisate.oromucosal' THEN 'SL' -- buprenorphine
        WHEN ofr.descr = 'filmsublingual.oromucosal' THEN 'SL' -- buprenorphine
        WHEN ofr.descr = 'vapour.oromucosal' THEN 'Inhal' -- nicotine inhalator - to match ATC
        WHEN ofr.descr LIKE '%.oral' THEN 'O'
        WHEN ofr.descr LIKE '%.gastroenteral' THEN 'O'
        WHEN ofr.descr LIKE '%.intravenous' THEN 'P'
        WHEN ofr.descr LIKE '%.intramuscular%' THEN 'P'
        WHEN ofr.descr LIKE '%.subcutaneous' THEN 'P'
        WHEN ofr.descr LIKE '%.intracavernous' THEN 'P'
        WHEN ofr.descr LIKE '%.intracardiac' THEN 'P'   
        WHEN ofr.descr LIKE '%.intracoronary' THEN 'P'  
        WHEN ofr.descr LIKE '%.vaginal' THEN 'V'
        WHEN ofr.descr LIKE '%.rectal' THEN 'R'
        WHEN ofr.descr LIKE '%.submucosalrectal' THEN 'R'
        WHEN ofr.descr LIKE '%.nasal' THEN 'N'
        WHEN ofr.descr LIKE '%.buccal' THEN 'SL'
        WHEN ofr.descr LIKE '%.sublingual' THEN 'SL'
        WHEN ofr.descr LIKE '%.transdermal' THEN 'TD'
        WHEN ofr.descr LIKE '%.urethral' THEN 'urethral'   
        WHEN ofr.descr LIKE '%.intravesical' THEN 'intravesical'
      END AS who_route
      FROM `ebmdatalab.dmd.vmp` vmp
    LEFT JOIN dmd.ont ont ON vmp.id = ont.vmp
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    LEFT JOIN dmd.droute droute ON vmp.id = droute.vmp
    LEFT JOIN dmd.route route ON droute.route = route.cd
    WHERE route.descr IS NOT NULL 
    AND ofr.descr IS NOT NULL
    AND ofr.descr NOT LIKE '%.auricular' -- eye drops don't seem to have DDDs
    AND ofr.descr NOT LIKE '%.bodycavity' -- ??
    AND ofr.descr NOT LIKE '%.cutaneous' -- no DDDs
    AND ofr.descr NOT LIKE '%.dental' -- mostly sodium fluoride toothpaste, chlorhexidine mouthwash - no obvious route match
    AND ofr.descr NOT LIKE '%.endocervical' -- no obvious route match
    AND ofr.descr NOT LIKE '%.endosinusial'-- no obvious route match
    AND ofr.descr NOT LIKE '%.endotracheopulmonary' -- possibly match on inhalation, but not much here with DDDs
    AND ofr.descr NOT LIKE '%.epidural' -- no obvious route match/ no relevent DDDs e.g. bupivacaine
    AND ofr.descr NOT LIKE '%.epilesional'-- no obvious route match
    AND ofr.descr NOT LIKE '%.extraamniotic'-- no obvious route match
    AND ofr.descr NOT LIKE '%.extracorporeal'-- no obvious route match
    AND ofr.descr NOT LIKE '%.gingival'-- no obvious route match
    AND ofr.descr NOT LIKE '%.haemodialysis'-- no obvious route match
    AND ofr.descr NOT LIKE '%.haemofiltration'-- no obvious route match
    AND ofr.descr NOT LIKE '%.implantation'-- no obvious route match. Not to be confused with subcutaneous implants
    AND ofr.descr NOT LIKE '%.infiltration'-- no obvious route match
    AND ofr.descr NOT LIKE '%.scalp'-- no obvious route match
    AND ofr.descr NOT LIKE '%.ophthalmic'-- no obvious route match
    AND ofr.descr NOT LIKE '%.opthalmic'-- typo for above
    AND ofr.descr NOT LIKE '%.intestinal'-- levodopa gels for Parkinson's disease
    AND ofr.descr NOT LIKE '%.intraarterial'-- contrast media. A few drugs but the probably also have a more common P route.
    AND ofr.descr NOT LIKE '%.intraarticular'-- limited interest
    AND ofr.descr NOT LIKE '%.intrabursal'-- limited interest
    AND ofr.descr NOT LIKE '%.intracameral'-- niche
    AND ofr.descr NOT LIKE '%.intracerebroventricular'-- niche
    AND ofr.descr NOT LIKE '%.intracholangiopancreatic'-- contrast media
    AND ofr.descr NOT LIKE '%.intradermal'-- limited interest - a few vaccines - probably no DDDs
    AND ofr.descr NOT LIKE '%.intradiscal'-- niche
    AND ofr.descr NOT LIKE '%.intraepidermal'-- skin prick tests
    AND ofr.descr NOT LIKE '%.intraglandular'-- contrast / Botox
    AND ofr.descr NOT LIKE '%.intralesional'-- contrast media. A few drugs but the probably also have a more common P route.
    AND ofr.descr NOT LIKE '%.intraocular'-- no obvious route match
    AND ofr.descr NOT LIKE '%.intraosseous'-- A few specific drugs but the probably also have a more common P route.
    AND ofr.descr NOT LIKE '%.intraperitoneal'-- no obvious route match
    AND ofr.descr NOT LIKE '%.intrapleural'-- niche
    AND ofr.descr NOT LIKE '%.intraputaminal'-- putamen = part of brain. Very niche - 1 product - gene therapy
    AND ofr.descr NOT LIKE '%.intrathecal'-- A few specific drugs but the probably also have a more common P route.
    AND ofr.descr NOT LIKE '%.intratumoral'-- niche
    AND ofr.descr NOT LIKE '%.intrauterine'-- no obvious route
    AND ofr.descr NOT LIKE '%.intraventricular cardiac'-- niche
    AND ofr.descr NOT LIKE '%.intravitreal'-- no obvious route. No DDDs for those of interest.
    AND ofr.descr NOT LIKE '%.iontophoresis'-- no obvious route
    AND ofr.descr NOT LIKE '%.linelock'-- no obvious route
    AND ofr.descr NOT LIKE '%.ocular'-- no obvious route
    AND ofr.descr NOT LIKE '%.periarticular'-- no obvious route
    AND ofr.descr NOT LIKE '%.peribulbar ocular'-- no obvious route
    AND ofr.descr NOT LIKE '%.perilesional'-- no obvious route
    AND ofr.descr NOT LIKE '%.perineural'-- no obvious route, niche
    AND ofr.descr NOT LIKE '%.peritumoral'-- no obvious route
    AND ofr.descr NOT LIKE '%.regionalperfusion'-- no obvious route
    AND ofr.descr NOT LIKE '%.subconjunctival'-- no obvious route
    AND ofr.descr NOT LIKE '%.submucosal'-- no obvious route, small number of products with a more common P route
    AND ofr.descr NOT LIKE '%.subretinal'-- no obvious route
    AND ofr.descr != 'gasinhalation.inhalation' -- medical gases
    AND ofr.descr != 'vapourinhalationliquid.inhalation' -- inhaled anaesthetics
    AND ofr.descr != 'vapourinhalation.inhalation' -- only 1 product - Benzoin compound tincture
    AND ofr.descr != 'impregnatedcigarette.inhalation' -- no DDDs
    AND ofr.descr NOT IN ('gargle.oromucosal', 'gel.oromucosal', 'homeopathicpillule.oromucosal', 'liquid.oromucosal', 'liquidspray.oromucosal', 'mouthwash.oromucosal', 'paste.oromucosal', 'homeopathictablet.oromucosal', 'pastille.oromucosal', 'solutionspray.oromucosal', 'suspension.oromucosal', 'suspensionspray.oromucosal', 'tabletmuco-adhesive.oromucosal', 'vapour.oromucosal') -- no DDDs
    ORDER BY who_route, dmd_r ASC
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'dmd_to_atc_route.csv')

# Use the cached_read function from the bq library to run the query.
dmd_to_atc_route = bq.cached_read(sql, csv_path=csv_path)

dmd_to_atc_route

,dmd_ofr,dmd_r,who_route
0,medicatedchewing-gum.oromucosal,oromucosal,Chewing gum
1,pressurizedinhalation.inhalation,inhalation,Inhal.aerosol
2,powderinhalation.inhalation,inhalation,Inhal.powder
3,solutionnebuliser.inhalation,inhalation,Inhal.solution
4,dispersionnebuliser.inhalation,inhalation,Inhal.solution
...,...,...,...
150,solutionirrigation.intravesical,intravesical,intravesical
151,solution.intravesical,intravesical,intravesical
152,stick.urethral,urethral,urethral
153,gel.urethral,urethral,urethral


The result of this is saved as a view within BigQuery - `dmd_to_atc_route`

### Checking for multiple routes
Now that we have a mapping, we can link this to VMP codes and look for any VMPs which have multiple associated routes - as this will cause issues joining to the DDD data.

In [5]:
sql = f"""
    SELECT 
      vmp.id AS vmp_id,
      vmp.nm AS vmp_nm,
      COUNT(DISTINCT routelookup.who_route) AS distinct_who_route_count
    FROM `ebmdatalab.dmd.vmp` vmp
    LEFT JOIN dmd.ont ont ON vmp.id = ont.vmp
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    INNER JOIN `dmd.vpi_to_atc_and_ddd` atclookup ON vmp.id = atclookup.VPID
    LEFT JOIN dmd.unitofmeasure uom ON atclookup.DDD_UOMCD = uom.cd
    INNER JOIN `scmd_dmd_views.dmd_to_atc_route` routelookup ON ofr.descr = routelookup.dmd_ofr
    INNER JOIN `scmd.ddd` ddd ON atclookup.ATC = ddd.atc_code AND routelookup.who_route = ddd.adm_code
    GROUP BY vmp.id, vmp.nm
    HAVING distinct_who_route_count > 1
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'multi_route_products.csv')

# Use the cached_read function from the bq library to run the query.
multi_route_products = bq.cached_read(sql, csv_path=csv_path)

multi_route_products

,vmp_id,vmp_nm,distinct_who_route_count
0,39709011000001102,Progesterone 400mg pessaries,2
1,41760311000001104,Colistimethate 1million unit powder for soluti...,2
2,41760411000001106,"Colistimethate 500,000unit powder for solution...",2
3,39708911000001106,Progesterone 200mg pessaries,2
4,36015811000001103,Phytomenadione 2mg/0.2ml solution for injectio...,2
5,4078411000001107,Paraldehyde 100% solution for injection 5ml am...,2
6,4638311000001106,Colistimethate 2million unit powder for soluti...,2
7,13821411000001101,Caffeine citrate 10mg/1ml solution for infusio...,2
8,20532611000001105,Caffeine citrate 20mg/1ml solution for infusio...,2


A review of the product information for these products show that the are licensed for use by multiple routes. A pragmatic decision can be made to decide the most likely route.

A table to manually map theses can be created like follows:

In [6]:
sql = f"""
    SELECT *
    FROM `ebmdatalab.chris.vmp_single_route_identifier`
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'vmp_single_route_identifier.csv')

# Use the cached_read function from the bq library to run the query.
vmp_single_route_identifier = bq.cached_read(sql, csv_path=csv_path)

vmp_single_route_identifier

,vmp_id,route
0,39709011000001102,V
1,39708911000001106,V
2,36015811000001103,P
3,4078411000001107,P
4,41760411000001106,P
5,4638311000001106,P
6,41760311000001104,P
7,20532611000001105,P
8,13821411000001101,P


With this fix we can create a mapping table of one-to-one dm+d route to WHO route.

In [7]:
sql = f"""
    SELECT DISTINCT
      vmp.id AS vmp_id,
      vmp.nm AS vmp_nm,
      COALESCE(sroute.route, routelookup.who_route) AS who_route,
      atclookup.ATC AS atc,
      atclookup.DDD as dmd_DDD,
      uom.descr as dmd_UOM,
      ddd.ddd AS who_ddd,
      ddd.unit_type AS who_UOM,
      ddd.ddd_comment AS who_ddd_comment
    FROM `ebmdatalab.dmd.vmp` vmp
    LEFT JOIN dmd.ont ont ON vmp.id = ont.vmp
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    INNER JOIN `dmd.vpi_to_atc_and_ddd` atclookup ON vmp.id = atclookup.VPID
    LEFT JOIN dmd.unitofmeasure uom ON atclookup.DDD_UOMCD = uom.cd
    LEFT JOIN chris.vmp_single_route_identifier sroute ON vmp.id = sroute.vmp_id
    INNER JOIN `scmd_dmd_views.dmd_to_atc_route` routelookup ON ofr.descr = routelookup.dmd_ofr
    INNER JOIN `scmd.ddd` ddd ON atclookup.ATC = ddd.atc_code AND COALESCE(sroute.route, routelookup.who_route) = ddd.adm_code
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'ddd_joined.csv')

# Use the cached_read function from the bq library to run the query.
ddd_joined = bq.cached_read(sql, csv_path=csv_path)

ddd_joined

,vmp_id,vmp_nm,who_route,atc,dmd_DDD,dmd_UOM,who_ddd,who_UOM,who_ddd_comment
0,42372711000001106,Carvedilol 3.125mg tablets,O,C07AG02,37.5,mg,37.5,mg,NaN
1,42372811000001103,Carvedilol 6.25mg tablets,O,C07AG02,37.5,mg,37.5,mg,NaN
2,42372511000001101,Carvedilol 12.5mg tablets,O,C07AG02,37.5,mg,37.5,mg,NaN
3,42372611000001102,Carvedilol 25mg tablets,O,C07AG02,37.5,mg,37.5,mg,NaN
4,42034011000001101,Cinnarizine 15mg tablets,O,N07CA02,90.0,mg,90.0,mg,NaN
...,...,...,...,...,...,...,...,...,...
9777,17020111000001106,"Epoetin theta 10,000units/1ml solution for inj...",P,B03XA01,1000.0,unit,1.0,TU,NaN
9778,17020211000001100,"Epoetin theta 2,000units/0.5ml solution for in...",P,B03XA01,1000.0,unit,1.0,TU,NaN
9779,17020511000001102,"Epoetin theta 30,000units/1ml solution for inj...",P,B03XA01,1000.0,unit,1.0,TU,NaN
9780,17020311000001108,"Epoetin theta 20,000units/1ml solution for inj...",P,B03XA01,1000.0,unit,1.0,TU,NaN


### Compared dm+d DDDs with ATC DDDs

We can use the dm+d DDDs to validate our matching (where they are given).
To do this we need to make sure both use the same units. We can add a new column `dmd_ddd_matched` which has the DDD value converted to the same unit as that used for the WHO DDD.

In [10]:
sql = f"""
    WITH matched_ddds AS (
      SELECT DISTINCT
      vmp.id AS vmp_id,
      vmp.nm AS vmp_nm,
      COALESCE(sroute.route, routelookup.who_route) AS who_route,
      atclookup.ATC AS atc,
      atclookup.DDD as dmd_DDD,
      uom.descr as dmd_UOM,
      CASE
        WHEN uom.descr = 'gram' AND ddd.unit_type = 'g' THEN atclookup.DDD
        WHEN uom.descr = 'mega unit' AND ddd.unit_type = 'MU' THEN atclookup.DDD
        WHEN uom.descr = 'mg' AND ddd.unit_type = 'mg' THEN atclookup.DDD
        WHEN uom.descr = 'mg' AND ddd.unit_type = 'g' THEN atclookup.DDD/1000
        WHEN uom.descr = 'mg' AND ddd.unit_type = 'mcg' THEN atclookup.DDD*1000
        WHEN uom.descr = 'microgram' AND ddd.unit_type = 'mcg' THEN atclookup.DDD
        WHEN uom.descr = 'microgram' AND ddd.unit_type = 'mg' THEN atclookup.DDD/1000
        WHEN uom.descr = 'tablet' AND ddd.unit_type = 'tablet' THEN atclookup.DDD
        WHEN uom.descr = 'unit' AND ddd.unit_type = 'U' THEN atclookup.DDD
        WHEN uom.descr = 'unit' AND ddd.unit_type = 'TU' THEN atclookup.DDD/1000
        WHEN uom.descr = 'unit' AND ddd.unit_type = 'MU' THEN atclookup.DDD/1000000
      END AS dmd_DDD_matched,
      ddd.unit_type AS dmd_UOM_matched,
      ddd.ddd AS who_ddd,
      ddd.unit_type AS who_UOM,
      ddd.ddd_comment AS who_ddd_comment
    FROM `ebmdatalab.dmd.vmp` vmp
    LEFT JOIN dmd.ont ont ON vmp.id = ont.vmp
    LEFT JOIN dmd.ontformroute ofr ON ont.form = ofr.cd
    INNER JOIN `dmd.vpi_to_atc_and_ddd` atclookup ON vmp.id = atclookup.VPID
    LEFT JOIN dmd.unitofmeasure uom ON atclookup.DDD_UOMCD = uom.cd
    LEFT JOIN chris.vmp_single_route_identifier sroute ON vmp.id = sroute.vmp_id
    INNER JOIN `scmd_dmd_views.dmd_to_atc_route` routelookup ON ofr.descr = routelookup.dmd_ofr
    INNER JOIN `scmd.ddd` ddd ON atclookup.ATC = ddd.atc_code AND COALESCE(sroute.route, routelookup.who_route) = ddd.adm_code
    )
    SELECT * FROM matched_ddds
    WHERE dmd_DDD_matched != who_ddd
    ORDER BY atc ASC
"""
# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'ddd_joined_unmatched.csv')

# Use the cached_read function from the bq library to run the query.
ddd_joined_unmatched = bq.cached_read(sql, csv_path=csv_path)

with pd.option_context('display.max_rows', None):
    display(ddd_joined_unmatched)

Downloading: 100%|█████████████████████████████████████████████████████████████|


,vmp_id,vmp_nm,who_route,atc,dmd_DDD,dmd_UOM,dmd_DDD_matched,dmd_UOM_matched,who_ddd,who_UOM,who_ddd_comment
0,35824211000001108,Budesonide 1mg orodispersible tablets sugar free,O,A07EA06,1.50,mg,1.500,mg,9.000,mg,None
1,40754711000001107,Budesonide 500microgram orodispersible tablets...,O,A07EA06,1.50,mg,1.500,mg,9.000,mg,None
2,42082711000001102,Glibenclamide 2.5mg tablets,O,A10BB01,10.00,mg,10.000,mg,7.000,mg,microcryst.substance
3,42082811000001105,Glibenclamide 5mg tablets,O,A10BB01,10.00,mg,10.000,mg,7.000,mg,microcryst.substance
4,8523711000001107,Glibenclamide 5mg/5ml oral solution,O,A10BB01,10.00,mg,10.000,mg,7.000,mg,microcryst.substance
5,8523911000001109,Glibenclamide 7.5mg/5ml oral solution,O,A10BB01,10.00,mg,10.000,mg,7.000,mg,microcryst.substance
6,8523811000001104,Glibenclamide 5mg/5ml oral suspension,O,A10BB01,10.00,mg,10.000,mg,7.000,mg,microcryst.substance
7,8524011000001107,Glibenclamide 7.5mg/5ml oral suspension,O,A10BB01,10.00,mg,10.000,mg,7.000,mg,microcryst.substance
8,12135811000001104,Glibenclamide 625micrograms/5ml oral suspension,O,A10BB01,7.00,mg,7.000,mg,10.000,mg,None
9,12135711000001107,Glibenclamide 625micrograms/5ml oral solution,O,A10BB01,7.00,mg,7.000,mg,10.000,mg,None


In [11]:
ddd_joined_unmatched.to_csv('ddd_joined_unmatched.csv')

---

### Next steps

#### Improve mapping
Identify reasons for unmatched rows.

Review and identify the relevent WHO DDD comments which will affect calculations - e.g. Amphotericin B - we need to split by liposomal and non-liposomal.

Look to manually map these issues

#### Look for unmatched

Identify rows with dm+d DDDs that we haven't matched to WHO DDD

Identify rows with WHO DDDs that we haven't matched to dm+d DDDs



#### Improve route matching

Review in more detail how we match and identify / review in more detail the borderline/complex cases.